In [1]:
import requests

def call_hdf5():
    try:
        # Make a GET request to the hdf5 service
        response = requests.get("http://localhost:5000/read")  # Replace "/endpoint" with the actual endpoint of your Flask app
        
        # Check if the request was successful
        if response.status_code == 200:
            return {"status": "success", "data": response.json()}
        else:
            return {"status": "error", "message": f"HTTP {response.status_code}: {response.text}"}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": str(e)}

In [2]:
print(call_hdf5())

{'status': 'success', 'data': [1, 2, 3, 4, 5]}


In [3]:
import julia
julia.install()

[ Info: Julia version info


Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  uname: Linux 6.12.13-100.fc40.x86_64 #1 SMP PREEMPT_DYNAMIC Sat Feb  8 17:10:01 UTC 2025 x86_64 unknown
  CPU: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz: 
              speed         user         nice          sys         idle          irq
       #1  3491 MHz      11280 s          4 s       3590 s     149491 s        449 s
       #2  3566 MHz      14336 s          2 s       3750 s     146741 s        431 s
       #3  3572 MHz      13238 s          1 s       3725 s     148001 s        415 s
       #4  3561 MHz      12304 s          4 s       3822 s     148860 s        402 s
       #5  3428 MHz      12633 s          2 s       3846 s     148574 s        404 s
       #6  3583 MHz       9750 s          2 s       3113 s     151733 s        578 s
       #7  3469 MHz      11423 s          4 s       4107 s     149152 s        676 s
 

[ Info: Julia executable: /opt/julia-1.9.3/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /home/alexmy/SGS/SGS.ai/.venv/bin/python
│     libpython: /usr/lib64/libpython3.12.so.1.0
│ Python:
│     python: /home/alexmy/SGS/SGS.ai/.venv/bin/python
└     libpython: /usr/lib64/libpython3.12.so.1.0


In [4]:
# meta_algebra.py

from julia import Main

# Load the sets32.jl file
Main.include("HllSets/src/HllSets.jl")

# using .HllSets

Main.using(".HllSets")

class HllSet:
    def __init__(self, P=10):
        """
        Initialize an HllSet with a given precision P.
        """
        self.P = P
        self.hll = Main.HllSet(P)  # Create a new HllSet in Julia

    def add(self, element):
        """
        Add an element to the HllSet.
        """
        # Use getattr to call the Julia function with '!'
        add_func = getattr(Main, "add!")
        add_func(self.hll, element)

    def count(self):
        """
        Estimate the cardinality of the HllSet.
        """
        return Main.count(self.hll)

    def union(self, other):
        """
        Perform a union with another HllSet.
        """
        result = Main.union(self.hll, other.hll)
        return HllSet.from_julia(result)

    def intersection(self, other):
        """
        Perform an intersection with another HllSet.
        """
        result = Main.intersect(self.hll, other.hll)
        return HllSet.from_julia(result)

    def difference(self, other):
        """
        Perform a difference with another HllSet.
        """
        result = Main.diff(self.hll, other.hll)
        return HllSet.from_julia(result)

    def complement(self, other):
        """
        Perform a complement operation with another HllSet.
        """
        result = Main.set_comp(self.hll, other.hll)
        return HllSet.from_julia(result)

    @classmethod
    def from_julia(cls, julia_hll):
        """
        Create a Python HllSet from a Julia HllSet.
        """
        hll = cls()
        hll.hll = julia_hll
        return hll

    def __repr__(self):
        return f"HllSet(P={self.P}, count={self.count()})"
    
# Create two HllSets
hll1 = Main.HllSets.HllSet()

In [5]:
hll1 = HllSet(10)
hll2 = HllSet(10)

hll1.add("apple")
hll1.add("banana")
hll1.add("cherry")

hll2.add("banana")
hll2.add("date")
hll2.add("elderberry")

# Perform union and intersection
result_union = hll1.union(hll2)
result_intersection = hll1.intersection(hll2)

print("Union:", result_union.count())
print("Intersection:", result_intersection.count())

Union: 5
Intersection: 2
